In [6]:
# Snip Investigations

import numpy as np
import pandas as pd
import pylab
import matplotlib.pyplot as plt
from scipy import stats
import inspect
import os
import csv 
import time
import sys
import glob
import pandas as pd

from tvb.simulator.lab import *
from tvb.simulator.plot.tools import *

# Input Simulation Pipeline
from SimulationPipeline import *
from useful_fns import *

matplotlib.rcParams['figure.figsize'] = (20.0, 10.0)

In [7]:
TseriesFile = glob.glob("do-not-track\\Old\\5_8_20\\LCycle_G[0.75*Tseries*_.csv")[1]
ScorrFile = glob.glob("do-not-track\\Old\\5_8_20\\LCycle_G[0.75*SCorr*_.csv")[1]
FCMFile = glob.glob("do-not-track\\Old\\5_8_20\\LCycle_G[0.75*FCM*_.csv")[1]
print(TseriesFile)

do-not-track\Old\5_8_20\LCycle_G[0.75]_MouseCortex_Tseries_20200805-031253_.csv


In [8]:
# These are Default Values for ParamsDict.  

# Empty dict
ParamsDict = { }

# Name of import file/zip - Which contains connectivity data.
ParamsDict["name"] = "MouseCortex"

# Calculate FC-FC for Mouse?
ParamsDict["FCFC"] = True

# Monitors or not?  (Aka BOLD or not?)
ParamsDict["BOLD"] = False

# Change to Binary Connectome? (If True, will change the connectome into binary)
ParamsDict["BINARY"] = True

# Snip is the the number of elements you wish to snip at the start to remove the initial condition effects.
ParamsDict["Snip"] = 10
# Note, if BOLD = False, Snip gets multiplied by 100, later in the SimulationPipeline code.  Not the actual dictionary element though.  

# Set the Random State/Seed for the Stochastic Integrator:
ParamsDict["RandState"] = 118

# Remove ith indexed region (7 corresponds to Frontal Pole Cerebral Cortex) - Give it a list if removing multiple regions.  Empty list removes nothing. 
ParamsDict["REMOVE"] = [7]

# Set Simulation Length:
ParamsDict["Simul_length"] = 1.2e5

# Set Linear Coupling Constant:
ParamsDict["G"] = np.array([0.47]) 

# Set integrator time step dt.
ParamsDict["dt"] = 0.1

# Set Additive Noise strength
ParamsDict["noise"] = np.array([0.000013])

# Set Wilson Cowan Model Parameters
ParamsDict["MODEL_c_ee"] = np.array([16.0])
ParamsDict["MODEL_c_ei"] = np.array([12.0])
ParamsDict["MODEL_c_ie"] = np.array([10.0])
ParamsDict["MODEL_c_ii"] = np.array([3.0])

# Model is now defined within SimulationPipeline.py
# However if you adjusting parameters other than these Coupling Parameters, then you need to redefine the model in this file per run.

ParamsDict["MODEL"] = models.WilsonCowan(c_ee=ParamsDict["MODEL_c_ee"],c_ei=ParamsDict["MODEL_c_ei"],c_ie=ParamsDict["MODEL_c_ie"] ,c_ii=ParamsDict["MODEL_c_ii"],
                                        a_e=numpy.array([1.0]),a_i=numpy.array([1.0]),b_e=numpy.array([4]),b_i=numpy.array([4]),tau_e=numpy.array([10.0]),
                                        tau_i=numpy.array([10.0])) 

# Params Dict tag (extra note tags for the name - Example to denote what's being changed/looped.)
ParamsDict["tag"] = ""

################################################################################################################################

# i is PBS_ARRAY_INDEX - Allows for creation of multiple jobs 
# i = int(sys.argv[1])

# Limit Cycle Params
ParamsDict["MODEL_c_ee"] = np.array([11.0])
ParamsDict["MODEL_c_ei"] = np.array([10.0])
ParamsDict["MODEL_c_ie"] = np.array([10.0])
ParamsDict["MODEL_c_ii"] = np.array([1.0])
#ParamsDict["G"] = np.array([i*0.05]) 

# Define the model if it is not yet defined.
if "MODEL" not in ParamsDict:
    ParamsDict["MODEL"] = models.WilsonCowan(c_ee=ParamsDict["MODEL_c_ee"],c_ei=ParamsDict["MODEL_c_ei"],c_ie=ParamsDict["MODEL_c_ie"] ,c_ii=ParamsDict["MODEL_c_ii"],
                                            a_e=numpy.array([1.0]),a_i=numpy.array([1.0]),b_e=numpy.array([2.8]),b_i=numpy.array([2.8]),tau_e=numpy.array([10.0]),
                                            tau_i=numpy.array([65.0]),) 

# Load the connectivity data from a zip file. 
con = connectivity.Connectivity.from_file(os.getcwd() +"/Connectomes/" + ParamsDict["name"] + ".zip")

# Now need to prepare the connectivity data accordingly.  Unfortuantely doesn't load eveyrthing in properly. May need to adjust con.undirected in future.

# Remove the ith row and column in centres, tract_lengths and weights. i.e. the specified region(s)
con.centres = np.delete(con.centres,ParamsDict["REMOVE"])
con.weights = np.delete(con.weights,obj=ParamsDict["REMOVE"],axis=0)
con.weights = np.delete(con.weights,obj=ParamsDict["REMOVE"],axis=1)
con.tract_lengths = np.delete(con.tract_lengths,obj=ParamsDict["REMOVE"],axis=0)
con.tract_lengths = np.delete(con.tract_lengths,obj=ParamsDict["REMOVE"],axis=1)

# Number of regions
con.number_of_regions = con.weights.shape[0]

# Change to Connectome to Binary if desired:
if ParamsDict["BINARY"]==True:
    con.weights = con.weights!=0

WARNING  File 'average_orientations' not found in ZIP.
WARNING  File 'cortical' not found in ZIP.
WARNING  File 'hemispheres' not found in ZIP.
WARNING  File 'areas' not found in ZIP.


In [19]:

# Functional Conenctivity MAtrix = Pearson Correlation.

TSeriesMatrix = np.genfromtxt(TseriesFile,delimiter="\t")[1:]

In [10]:

Snip = 1

while Snip < 1e6:
    print(Snip)

    FCM = np.corrcoef(TSeriesMatrix[:,:len(TSeriesMatrix[0])-Snip])

    # Set diagonals to NaN
    FCM1 = FCM
    np.fill_diagonal(FCM1,np.nan)

    # Comparing SC vs FC with Spearman Corr  (Known as SCFC)
    # Check if SCM is symmetric: 
    SCM = con.weights
    Sym_check = numpy.allclose(SCM, SCM.T,equal_nan=True)

    if Sym_check == True:
        #It is a symmetric SCM, so only use upper triangles
        # Grab Upper triangles
        FCM_Upper = FCM[np.triu_indices(FCM.shape[0], k = 1)]
        SCM_Upper = con.weights[np.triu_indices(con.weights.shape[0], k = 1)]

    elif Sym_check == False:
        # If SCM is not symmetric, need to calcualte spearman corr for entire matrix.
        # Set Diagonal to Nans
        np.fill_diagonal(SCM,np.nan)
        # Remove all Nans for SCM and FCM
        SCM_Upper = SCM[~numpy.isnan(SCM)]
        FCM_Upper = FCM1[~numpy.isnan(FCM1)]

    # Spearman Correlation
    Scorr = stats.spearmanr(a=FCM_Upper,b=SCM_Upper)
    print("SCFC = ",Scorr)

    # Calculate FC-FC score for mouse if requested.
    if ParamsDict["FCFC"] == True:
        # FCM_exp
        FCM_exp = np.genfromtxt('FCM_MouseExperimental.csv',delimiter = "\t")
        # Set diagonals to NaN
        np.fill_diagonal(FCM_exp,np.nan)

        # Remove the ith row and column in FCM (i.e. the specified region)
        FCM_exp = np.delete(FCM_exp,obj=ParamsDict["REMOVE"],axis=0)
        FCM_exp = np.delete(FCM_exp,obj=ParamsDict["REMOVE"],axis=1)

        # Comparing FC_experimental Vs FC_Simulation with Spearman Correlation

        FCM_Exp_U = FCM_exp[np.triu_indices(FCM_exp.shape[0], k = 1)]  
        FCM_Upper = FCM[np.triu_indices(FCM.shape[0], k = 1)]

        # FC-FC Spearman Correlation
        FCFC = stats.spearmanr(a=FCM_Exp_U,b=FCM_Upper)
        print("FCFC = ",FCFC)
        # Concancatanate to end of Scorr output file. 
        Scorr = Scorr + FCFC

    Snip = Snip * 10

1
SCFC =  SpearmanrResult(correlation=0.39001655130012575, pvalue=1.2354801358988211e-49)
FCFC =  SpearmanrResult(correlation=0.6129895629303821, pvalue=5.7513349554580164e-70)
10
SCFC =  SpearmanrResult(correlation=0.3900971456589239, pvalue=1.175791982143924e-49)
FCFC =  SpearmanrResult(correlation=0.613030062899865, pvalue=5.601068427039144e-70)
100
SCFC =  SpearmanrResult(correlation=0.3900678437055767, pvalue=1.1971537493180992e-49)
FCFC =  SpearmanrResult(correlation=0.6132501520018289, pvalue=4.850195735306111e-70)
1000
SCFC = SpearmanrResult(correlation=0.39038286360334234, pvalue=9.863772978751427e-50)
FCFC =  SpearmanrResult(correlation=0.6144452057653352, pvalue=2.215477685245727e-70)
10000
SCFC =  SpearmanrResult(correlation=0.3864635172803344, pvalue=1.0813446646225768e-48)
FCFC =  SpearmanrResult(correlation=0.6161577817065863, pvalue=7.165704114663534e-71)
100000
SCFC =  SpearmanrResult(correlation=0.35912096383650566, pvalue=8.039788142483966e-42)
FCFC =  SpearmanrResul

We thus see, that as we increase Snip size, FCFC score doesn't really change much. 
Until we snip 1e5 of data (which is like 5/6 of the data) and the usual, since we are taking the correlation of a shorter data length, our error bars indicate that the result we get is worse. 
This salso uggests, that the snip we have so far already allows the system to reach equilibrium if it ever does. 